# Road Lane Detection
## (Self-Guided Project)

## by Justin Sierchio

In this project, we will be looking at road lane detections.

This data is in .csv file format and is from GeeksforGeeks at: https://www.geeksforgeeks.org/opencv-real-time-road-lane-detection/. More information related to the dataset can be found at the same link.

Note: this is a self-guided project following the tutorial provided the contributors at GeeksforGeeks.

## Notebook Initialization

In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

print('Initial libraries loaded into workspace!')

Initial libraries loaded into workspace!


## Description

Specifically, we are going to look at the road lanes where autonomous cars will run. We will be trying to detect white markings for the lanes by masking the rest of the frame. One way to do this is measure the change in intensity (light or dark) using derivatives (where large derivatives would indicate a substantial change).

## Algorithm Development

Let's begin by construction of a function called a 'canny edge' detector.

In [15]:
# Create a Function to complete canny edge detection
def canny_edge_detector(image):
      
    # Convert the image color to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
      
    # Reduce noise from the image
    blur = cv2.GaussianBlur(gray_image, (5, 5), 0) 
    canny = cv2.Canny(blur, 50, 150)
    return canny

Next, we will mask the canny image after we have found our region of interest (ROI):

In [16]:
# Define a region of interest and mask the canny image
def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
        [(200, height), (1100, height), (550, 250)]
        ])
    mask = np.zeros_like(image)
      
    # Fill poly-function deals with multiple polygon
    cv2.fillPoly(mask, polygons, 255) 
      
    # Bitwise operation between canny image and mask image
    masked_image = cv2.bitwise_and(image, mask) 
    return masked_image

Now, we want to define our road lane's coordinates.

In [18]:
# Create a Function to define road lane coordinates
def create_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1 * (3 / 5))
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1, y1, x2, y2])

Depending on what side of the road we are looking at, we need to distinguish between left and right road lanes. This step is needed because the derivatives will have slopes with opposite signs.

In [19]:
# Left-Right lane distinction function
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
          
        # It will fit the polynomial and the intercept and slope
        parameters = np.polyfit((x1, x2), (y1, y2), 1) 
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
              
    left_fit_average = np.average(left_fit, axis = 0)
    right_fit_average = np.average(right_fit, axis = 0)
    left_line = create_coordinates(image, left_fit_average)
    right_line = create_coordinates(image, right_fit_average)
    return np.array([left_line, right_line])

Next, let's fit the coordinates to our actual image.

In [20]:
# Fit coordinates to image and display line
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

Now that all the functions have been constructed, we can run the code.

## Algorithm Execution

Let's load a video and run the algorithm. The video we used can be downloaded for free at https://github.com/rslim087a/road-video.

In [ ]:
# Path of dataset directory
cap = cv2.VideoCapture("test2.mp4") 
while(cap.isOpened()):
    _, frame = cap.read()
    canny_image = canny_edge_detector(frame)
    cropped_image = region_of_interest(canny_image)
      
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi / 180, 100, 
                            np.array([]), minLineLength = 40, 
                            maxLineGap = 5) 
      
    averaged_lines = average_slope_intercept(frame, lines) 
    line_image = display_lines(frame, averaged_lines)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1) 
    cv2.imshow("results", combo_image)
      
    # When the below two will be true and will press the 'q' on
    # our keyboard, we will break out from the loop
      
    # # wait 0 will wait for infinitely between each frames. 
    # 1ms will wait for the specified time only between each frames
    if cv2.waitKey(1) & 0xFF == ord('q'):      
        break

# close the video file
cap.release() 
  
# destroy all the windows that is currently on
cv2.destroyAllWindows() 

For the purposes of coding, the result can be seen at https://www.geeksforgeeks.org/opencv-real-time-road-lane-detection/.

# Conclusion

The goal of this self-guided project (via GeeksforGeeks) was to explore and detect road lanes on highways. In this project, we were able to upload a real-world dataset, run a machine learning algorithm and present our results. It is the author's hope that others find this exericse useful. Thanks for reading!